In [1]:
#Bibliotecas
import pandas as pd
from pandas import DataFrame
import numpy as np
#np.random.seed(600)
#np.random.seed(700)
random_seed = 600
np.random.seed(random_seed)
import screed # Librería para leer archivos en FASTA/FASTQ
import itertools
import matplotlib.pyplot as plt
import random 
random.seed(random_seed)

from numpy import linalg # SVD

import os
os.environ['PYTHONHASHSEED'] = '0'

import tensorflow as tf
tf.random.set_seed(random_seed)

import keras

from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten

from sklearn.model_selection import train_test_split

%matplotlib inline
#---------------

In [2]:
def show_accuracy_model(history, skip):
    # Plot training & validation accuracy values
    #plt.plot(history.history['accuracy'])
    #plt.plot(history.history['val_accuracy'])
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    plt.plot(np.arange(skip, len(accuracy), 1), accuracy[skip:])
    plt.plot(np.arange(skip, len(val_accuracy), 1), val_accuracy[skip:])
    plt.title('Model Train: Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [3]:
def show_loss_model(history, skip):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    #plt.plot(history.history['loss'])
    #plt.plot(history.history['val_loss'])
    plt.plot(np.arange(skip, len(loss), 1), loss[skip:])
    plt.plot(np.arange(skip, len(val_loss), 1), val_loss[skip:])
    plt.title('Model Train: Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='best')
    plt.show()

In [4]:
def add_class(df):
    
    class_ACP = []
    class_noACP = []
    
    rows_model = df.shape[0]
    
    for x in range(int(rows_model/2)):
        class_ACP.append(1)
    
    for x in range(int(rows_model/2)):
        class_noACP.append(0)
        
    df['Class'] = class_ACP + class_noACP
    
    return df

In [5]:
def change_index(df):
    #Cambiar indices de filas y columnas por valores numéricos
    total_rows_df = df.shape[0]
    df.index = np.arange(0, total_rows_df)

    total_columns_df = df.shape[1]
    df.columns = np.arange(0, total_columns_df)
    
    return df

In [6]:
def save(self):
    if not os.path.exists('./weights'):
        os.mkdir('./weights')
    else:
        #autoencoder.encoded_imgs.save('./weights/encoder_weights.h5')
        #autoencoder.predicted.save('./weights/decoder_weights.h5')
        self.save('./weights/ae_weights.h5') #Almacenar pesos del modelo

In [7]:
def show_accuracy(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    #plt.plot(hist_Deep_AE.history['val_accuracy'])
    plt.title('Model Train: Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train'], loc='upper left')
    plt.show()

In [8]:
def show_error(history, skip):
    loss = history.history['loss']
    #val_loss = history.history['val_loss']
    plt.plot(np.arange(skip, len(loss), 1), loss[skip:])
    #plt.plot(np.arange(skip, len(loss), 1), val_loss[skip:])
    plt.title('Model Train: Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='best')
    plt.show()

In [9]:
def build_deep_autoencoder(one_hot_sequences_ACP, one_hot_sequences_noACP):
    
    df_onehot_ACP = pd.DataFrame(one_hot_sequences_ACP) #Crear Dataframe de OneHot ACP
    df_onehot_noACP = pd.DataFrame(one_hot_sequences_noACP) #Crear Dataframe de OneHot noACP
    
    dataframes = [df_onehot_ACP, df_onehot_noACP]
    df = pd.concat(dataframes)
    
    filas_total_df = df.shape[0]
    df.index = np.arange(0, filas_total_df) #Corregir los índices del nuevo dataframe
    
    dataset = df.values #Almacenar solo los valores en un nuevo dataframe
    
    in_out_dim = dataset.shape[1]  #Obtener el largo de los vectores leidos como input del modelo
    
    #Este es el tamaño de las representaciones encodeadas en el modelo
    encoding_dim = 32 # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
    #Placegoholder de entrada
    input_seq = Input(shape=(in_out_dim,))
    #"encoded" es la representación encodeada de la entrada
    encoded = Dense(128, activation='relu')(input_seq)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    #"decoded" es la reconstrucción con pérdidas de la entrada (específicamente de la cada encodeada)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(in_out_dim, activation='sigmoid')(decoded)
    
    #encoded = Dense(256, activation='relu')(input_seq)
    #encoded = Dense(128, activation='relu')(encoded)
    #encoded = Dense(64, activation='relu')(encoded)

    #decoded = Dense(128, activation='relu')(encoded)
    #decoded = Dense(256, activation='relu')(decoded)
    #decoded = Dense(encoding_dim, activation='sigmoid')(decoded)
    
    #Este modelo mapea una entrada a su reconstrucción
    autoencoder = Model(input_seq, decoded)
    #Este modelo mapea una entrada a una representación encodeada
    encoder = Model(input_seq, encoded)
    
    # Crear un placeholder para una entrada de 32 dimensiones
    encoded_input = Input(shape=(encoding_dim,))
    #Recuperar la capa encodeada del modelo (layers[-3]) hasta llegar a la última de la parte de decodificación
    decoder_layer = autoencoder.layers[-3](encoded_input)
    decoder_layer = autoencoder.layers[-2](decoder_layer)
    decoder_layer = autoencoder.layers[-1](decoder_layer)
    #Crear el modelo de decodificación
    decoder = Model(encoded_input, decoder_layer)
    
    return df, dataset, autoencoder, encoder, decoder
    

In [10]:
def read_descriptors(filepath_ACP_AAC,filepath_nonACP_AAC, filepath_ACP_DPC, filepath_noACP_DPC, filepath_ACP_PCP, filepath_noACP_PCP):
    
    #Leer Ficheros
    #AAC
    AAC_ACP = pd.read_csv(filepath_ACP_AAC, sep="\t", header=0)
    AAC_noACP = pd.read_csv(filepath_nonACP_AAC, sep="\t", header=0)
    #DPC
    DPC_ACP = pd.read_csv(filepath_ACP_DPC, sep="\t", header=0)
    DPC_noACP = pd.read_csv(filepath_noACP_DPC, sep="\t", header=0)
    #PCP
    PCP_ACP = pd.read_csv(filepath_ACP_PCP, sep="\t", header=0, error_bad_lines=False)
    PCP_noACP = pd.read_csv(filepath_noACP_PCP, sep="\t", header=0, error_bad_lines=False)
    
    #Eliminar primera fila (nombre de secuencias. ej: seq_1...)
    #ACC
    AAC_ACP = AAC_ACP.drop("#", axis=1)
    AAC_noACP = AAC_noACP.drop("#", axis=1)
    #DPC
    DPC_ACP = DPC_ACP.drop("#", axis=1)
    DPC_noACP = DPC_noACP.drop("#", axis=1)
    #PCP
    PCP_ACP = PCP_ACP.drop("FASTA_NAME", axis=1)
    PCP_noACP = PCP_noACP.drop("FASTA_NAME", axis=1)

    #Unir Conjuntos de características por la derecha: AAC - DPC -PCP
    df_ACP_feature = AAC_ACP.join(DPC_ACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    df_noACP_feature = AAC_noACP.join(DPC_noACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)

    df_ACP_feature = df_ACP_feature.join(PCP_ACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    df_noACP_feature = df_noACP_feature.join(PCP_noACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)

    #Concatenar dataframes de clase positiva con negativa
    #dataframes_features = [df_ACP_feature, df_noACP_feature]
    #df_features = pd.concat(dataframes_features)

    #(1516, 660)
    #print(df_features.shape)
    
    #Cambiar indices de filas y columnas por valores numéricos
    #total_rows_df = df_features.shape[0]
    #df_features.index = np.arange(0, total_rows_df)

    #total_columns_df = df_features.shape[1]
    #df_features.columns = np.arange(0, total_columns_df)
    
    #return df_features
    
    return df_ACP_feature, df_noACP_feature

In [11]:
def one_hot_encoding(sequences,alphabet,padding):
    
    one_hot_sequence_list = []
    alphabet_one_hot = {}
    
    alphabet_lenght = len(alphabet)
    
    max_length = max(len(x) for x in sequences)
        
    for i in range(len(alphabet)): #Crear los array de dimension 20 llenados con cero para cada una de las letras del abecedario de AA
        alphabet_one_hot[alphabet[i]] = [0]*alphabet_lenght #Agregar un arreglo (20-dimensional) para cada letra (posición i dentro del diccionario "alphabet")
        alphabet_one_hot[alphabet[i]][i] = 1 #Agregar un 1 en la posición que corresponda a cada letra en el arreglo creado anteriormente

    for i in range((len(sequences))): #Recorre cada una de las secuencias. 1516 en total (758 ACP - 758 noACP)
        one_hot_sequence = [] #Arreglo de Arreglos, que almacena cada una de las letras pero en su versión One Hot
        for j in range(len(sequences[i])): #Recorrer cada una de las letras (i) de una secuencia (secuences[i])
            #Dependiendo de la letra leida, agrega el arreglo de 20 dimensiones creado anteriormente que corresponde a dicha letra en cuestión
            #A = [1,0,...,0], C = [0,1,0,...,0], etc.
            one_hot_sequence = one_hot_sequence + alphabet_one_hot[sequences[i][j]]
        
        if (max_length<=padding): # Hacer Padding unicamente si el valor es <= al valor de la secuencia más larga
            for k in range(padding-len(sequences[i])):
                one_hot_sequence = one_hot_sequence + [0]*alphabet_lenght #Llenar los espacios faltantes
        one_hot_sequence_list.append(one_hot_sequence)

    #return np.array(one_hot_sequence_list)
    return one_hot_sequence_list

In [12]:
def svd(sequences, sparse_matrix, alphabet):
    from numpy import linalg
    
    sparse_matrix_SVD = []
    tri_feature = [0] * len(alphabet) # Largo del vector resultante pero también es la dimensión de U (filasxfilas de M original)
    
    for i in range(len(sparse_matrix)):
        sequence_lenght = len(sequences[i])
        U, s, Vt = linalg.svd(sparse_matrix[i])
        for j in range(len(s)):
            tri_feature = tri_feature + U[j] * s[j] / sequence_lenght # Multiplica cada una de las filas de U por los elementos de s y entrega la sumatoria
        sparse_matrix_SVD.append(tri_feature)
    return sparse_matrix_SVD

In [13]:
def create_sparse_matrix(alphabet_kmers,sequences_kmers):
    
    list_sparse_matrix = [] #Lista que contiene todas las sparse matrix generadas

    for i in range(len(sequences_kmers)): 

        sparse_matrix = [] # Inicializacion de la sparse matrix de cada secuencia

        for x in range(len(alphabet_kmers)):
            sparse_matrix.append([]) # Cantidad de filas que contendrá la matriz (numero de letras del alfabeto ^ K)

        for j in range(len(sequences_kmers[i])): # Avanza entre los k-mers de una misma secuencia
            for k in range(len(alphabet_kmers)): # Compara cada k-mers con las distintas posibilidades de combinaciones del alfabeto
                if (sequences_kmers[i][j] == alphabet_kmers[k]): # Comparacion de los K-mers de cada secuencia con las posibles combinaciones del alfabeto
                    sparse_matrix[k].append(1) #1: si son iguales
                else:
                    sparse_matrix[k].append(0)#0: si no son iguales
        list_sparse_matrix.append(sparse_matrix) # Agregación de cada sparse matrix generada para cada secuencia a la lista
    return list_sparse_matrix

In [14]:
def reduced_sequence(all_sequences, alphabet):
    all_sequences_reduced = [] # Almacena todas las secuencias codificadas y la devuelve a la función principal
    number_groups = len(alphabet.keys()) # Cantidad de Grupos que contiene el nuevo alfabeto
    number_sequences = len(all_sequences) # Cantidad de Secuencias a Codificar
    for i in range(number_sequences): # Recorrer todas las secuencias
        count = 0 # Contador = 0: Se está examinando la primera letra de la secuencia; Contador !=0: Se extá examinando cualquier otra letra
        first_letter_found = False # Verifica si se ha encontrado la primera letra dentro del diccionario
        next_letter_found = False # Verifica si se ha encontrado la letra "i" dentro de la secuencia a codificar
        index = 1 # Indice que recorre las "keys" del diccionario que contiene los grupos de letras (alphabet)
        for j in range(len(all_sequences[i])): # Recorre cada una de las letras de la secuencia seleccionada
            if (count==0):
                while ((first_letter_found==False) and (index <= number_groups)): # Asignar la primera letra codificada del alfabeto a la nueva secuencia
                    if (alphabet[index].count(all_sequences[i][j])): # Verifica si la letra se encuentra dentro del diccionario
                        all_sequences_reduced.append(str(index))
                        first_letter_found = True # Se encontró la primera letra de la secuencia a codificar
                        count = count + 1
                    else:
                        index = index + 1 # No se encontró la primera letra, se busca en el siguiente grupo (index + 1)
            else:
                index = 1
                while ((next_letter_found==False) and (index <= number_groups)):
                    if (alphabet[index].count(all_sequences[i][j])): 
                        aux = all_sequences_reduced[i] + str(index)
                        all_sequences_reduced[i] = aux
                        next_letter_found = True
                    else:
                        index = index + 1
            next_letter_found = False         
    return all_sequences_reduced

In [15]:
def sort_dictionary(dictionary):
    #groups = alphabet_reduction["DipoleMoment"].values()
    #Dictionary = key1:[values1], key2:[values2],..., keyn:[valuesn]
    sorted_list = []
    key_list = [] #Lista que almacena las nuevas Key ordenadas del Diccionario ordenado
    dic_result = {}
    for lenght in range(len(dictionary)): #Cantidad de grupos en esa codificación = len(dictionary)
        key_list.append(lenght+1) #Se agrega cada una de las key ordenadas a la lista
    for dic in dictionary: #Extrae cada uno de los valores (dic) que corresponden a vectores en el diccionario
        sorted_list.append(sorted(dic)) #Ordena los valores dentro de los vectores en orden de menor a mayor (letra o número)
    sorted_list = sorted(sorted_list) #Una vez ordenados los valores dentro de los vectores, se ordenan ahora los vectores
    index = 0
    for values in sorted_list:
        dic_result[int(key_list[index])] = values #Asignamos cada unos de los vectores (values) ordenados a cada una de las key ordenadas
        index = index + 1
    return dic_result #Se devuelve un nuevo diccionario ordenado

In [16]:
def build_kmers(sequence, ksize):
    kmers = []
    n_kmers = len(sequence) - ksize + 1 # Cantidad de K-mers totales por secuencia = L - k + 1
    
    for i in range(n_kmers):
        kmer = sequence[i:i + ksize] #Selecciona la cantidad de letras especificadas por "ksize"
        kmers.append(kmer) #Cada una de las posiciones almacena un k-mer de tamaño k

    return kmers

In [17]:
def read_kmers_from_sequences(sequences, ksize):
    all_kmers = []
    
    for sequence in sequences:
        if (len(sequence) >= ksize): #El valor de K siempre debe ser menor al largo de las secuencias
            kmers = build_kmers(sequence, ksize)
            #all_kmers += kmers
            all_kmers.append(kmers)   
    return all_kmers

In [18]:
def read_all_sequences(filename):
    all_sequences = []
    for record in screed.open(filename):
        sequence = record.sequence
        all_sequences.append(sequence)   
    return all_sequences

In [19]:
def alphabet_operations(K, encoded_word):
    
    alphabet = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] #B-J-O-U-X-Z NO PRESENTES
    
    #Codificacion del alfabeto de aa en grupos definido por sus características fisico-químicas
    alphabet_reduction = { "Hydrophobicity" : {1: ['R','K','E','D','Q','N'], 2:['P','H','Y','G','A','S','T'],
                                           3:['M','F','W','C','V','L','I']},
                         "Polarizability": {1: ['G','A','S','D','T'], 2:['E','Q','I','L','C','P','N','V'], 
                                            3:['R','Y','W','K','M','H','F']},
                         "VanWaals": {1: ['G','A','S','C','T','P','D'], 2:['N','V','E','Q','I','L'],
                                      3:['F','R','Y','W','M','H','K']},
                         "Polarity": {1: ['C','M','V','Y','L','I','F','W'], 2:['P','A','T','G','S'],
                                      3:['K','N','E','D','H','Q','R']},
                         "DipoleMoment": {1: ['A','G','V'], 2: ['I','L','F','P'], 3: ['Y','M','T','S'], 
                                           4: ['H','N','Q','W'], 5: ['R','K'], 6:['D','E'], 7: ['C']}           
    }
    
    #Creación de K-mers presentes en el alfabeto de AA
    alphabet_kmers = [''.join(i) for i in itertools.product(alphabet, repeat = K)]
    alphabet_kmers = sorted(alphabet_kmers)
    
    #Codificacion del alfabeto
    alphabet_reduced = reduced_sequence(alphabet, alphabet_reduction[encoded_word])
    
    # Eliminar "letras" repetidas
    alphabet_reduced = list(dict.fromkeys(alphabet_reduced))
    alphabet_kmers_reduced = [''.join(i) for i in itertools.product(alphabet_reduced, repeat = K)]
    alphabet_kmers_reduced = sorted(alphabet_kmers_reduced )
    
    return alphabet, alphabet_reduction, alphabet_kmers, alphabet_reduced, alphabet_kmers_reduced

In [28]:
def ACP():
    
    #Parametros a Modificar
    K = 3
    # "Hydrophobicity" "Polarizability" "VanWaals" "Polarity" "DipoleMoment"
    encoded_word = "Polarizability"
    #encoded_word = "Hydrophobicity"
    #encoded_word = "VanWaals"
    #encoded_word = "Polarity"
    #encoded_word = "DipoleMoment"
    
    alphabet, alphabet_reduction, alphabet_kmers, alphabet_reduced, alphabet_kmers_reduced = alphabet_operations(K, encoded_word)
    
    # Largo Combinación de Alfabeto = (Cantidad de Letras en el Alfabeto)^(K)
    #len(alphabet_kmers)
    
    #path_ACP = 'Databases/ACP.fasta'
    #path_noACP = 'Databases/noACP.fasta'
    path_ACP = '../Databases/ACP.fasta'
    path_noACP = '../Databases/noACP.fasta'
    
    # Leyendo Secuencias desde archivo fasta y almacenando secuencias
    all_sequences_ACP = read_all_sequences(path_ACP) # ACPs
    all_sequences_noACP = read_all_sequences(path_noACP) # noACPs
    #print('Cantidad de Secuencias Leidas: ACP ',len(all_sequences_ACP), '- noACP: ',len(all_sequences_noACP))
    
    alphabet_reduction[encoded_word] = sort_dictionary(alphabet_reduction[encoded_word].values())
    
    #Codificación de Secuencias
    all_sequences_ACP_reduced = reduced_sequence(all_sequences_ACP, alphabet_reduction[encoded_word])
    all_sequences_noACP_reduced = reduced_sequence(all_sequences_noACP, alphabet_reduction[encoded_word])
    
    #alphabet_reduction[encoded_word]
    
    #Creando k-mers reducidos
    sequences_kmers_ACP_reduced = read_kmers_from_sequences(all_sequences_ACP_reduced, K) # ACPs
    sequences_kmers_noACP_reduced = read_kmers_from_sequences(all_sequences_noACP_reduced, K) # noACPs
    
    #----------------------------------------------------------------------------------------------------------------------
    #K-MERS SPARSE MATRIX
    
    #Sparse Matrix co alfabeto reducido
    list_sparse_matrix_ACP_reduced = create_sparse_matrix(alphabet_kmers_reduced, sequences_kmers_ACP_reduced)
    list_sparse_matrix_noACP_reduced = create_sparse_matrix(alphabet_kmers_reduced, sequences_kmers_noACP_reduced)
    
    #Transformar las Sparse Matrix en vectores mediante el algoritmo SVD
    sparse_matrix_SVD_ACP = svd(all_sequences_ACP_reduced, list_sparse_matrix_ACP_reduced, alphabet_kmers_reduced)
    sparse_matrix_SVD_noACP = svd(all_sequences_noACP_reduced, list_sparse_matrix_noACP_reduced, alphabet_kmers_reduced)
    
    print('ACP- Cantidad de Sparse-Matrix: ', len(sparse_matrix_SVD_ACP), '- Largo Sparse Matrix: ', len(sparse_matrix_SVD_ACP[0]))
    print('noACP- Cantidad de Sparse-Matrix: ', len(sparse_matrix_SVD_noACP), '- Largo Sparse Matrix: ', len(sparse_matrix_SVD_noACP[0]))
    
    #----------------------------------------------------------------------------------------------------------------------
    #One Hot Encoding
    
    #One Hot Encoding de secuencias originales
    one_hot_sequences_ACP = one_hot_encoding(all_sequences_ACP, alphabet, 50)
    one_hot_sequences_noACP = one_hot_encoding(all_sequences_noACP, alphabet, 50)
    
    #np.array(one_hot_sequences_ACP).shape
    
    ##df_model_ACP = sparse_matrix_SVD_ACP.join(one_hot_sequences_ACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    ##df_model_noACP = sparse_matrix_SVD_noACP.join(one_hot_sequences_noACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    
    #-----------------------------------------------------------------------------------------------
    #Feature Extraction: AAC - DPC
    
    #AAC
    filepath_ACP_AAC = "../Databases/AAC_ACP.tsv" #ACP AAC
    filepath_nonACP_AAC = "../Databases/AAC_noACP.tsv" #nonACP AAC
    #DPC
    filepath_ACP_DPC = "../Databases/DPC_ACP.tsv" #ACP AAC
    filepath_noACP_DPC = "../Databases/DPC_noACP.tsv" #nonACP AAC
    #PCP
    filepath_ACP_PCP = "../Databases/PCP_ACP_fixed.txt" #ACP AAC
    filepath_noACP_PCP = "../Databases/PCP_noACP_fixed.txt" #nonACP AAC
    
    df_ACP_feature, df_noACP_feature = read_descriptors(filepath_ACP_AAC,filepath_nonACP_AAC, filepath_ACP_DPC, 
                                                        filepath_noACP_DPC, filepath_ACP_PCP, filepath_noACP_PCP)
    
    ##df_model_ACP = df_model_ACP.join(df_ACP_feature, how='right', lsuffix='', rsuffix='_feature', sort=False)
    ##df_model_noACP = df_model_noACP.join(df_noACP_feature, how='right', lsuffix='', rsuffix='_feature', sort=False)
    
    #-------------------------------------------------------------------------------------------------------
    #Deep Autoencoder
    
    df, dataset, autoencoder, encoder, decoder = build_deep_autoencoder(one_hot_sequences_ACP, one_hot_sequences_noACP)
    
    #autoencoder.summary()
    #encoder.summary()
    #decoder.summary()
    
    #autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    epochs = 1000
    batch_size = 256
    
    skip = 0
    
    hist_Deep_AE = autoencoder.fit(dataset, dataset,
                    epochs = epochs,
                    batch_size = batch_size,
                    shuffle = False, verbose=0)
    
    #show_error(hist_Deep_AE, 0) #skip = numero de epochs que salta
    #show_accuracy(hist_Deep_AE)
    
    #LAYERS INFO
    #for layer in autoencoder.layers: 
      #print(layer.get_config())
        
    encoded_seq = encoder.predict(dataset)
    decoded_seq = decoder.predict(encoded_seq)
    predicted = autoencoder.predict(dataset)
    
    #-------------------------------------------------------------------------------------------------------
    #Create Input Dataframe for the CNN model
    
    #len(sparse_matrix_SVD_ACP[0]) + len(one_hot_sequences_ACP[0]) + df_ACP_feature.shape[1] + len(encoded_seq[0])
    
    df_sparse_ACP = DataFrame(sparse_matrix_SVD_ACP)
    df_sparse_noACP = DataFrame(sparse_matrix_SVD_noACP)
    
    df_onehot_ACP = DataFrame(one_hot_sequences_ACP)
    df_onehot_noACP = DataFrame(one_hot_sequences_noACP)
    
    df_model_ACP = df_sparse_ACP.join(df_onehot_ACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    df_model_noACP = df_sparse_noACP.join(df_onehot_noACP, how='right', lsuffix='', rsuffix='_onehot', sort=False)
    
    #df_model_ACP = df_model_ACP.join(df_ACP_feature, how='right', lsuffix='', rsuffix='_feature', sort=False)
    #df_model_noACP = df_model_noACP.join(df_noACP_feature, how='right', lsuffix='', rsuffix='_feature', sort=False)
    
    df_model = pd.concat([df_model_ACP, df_model_noACP])
    
    df_model = change_index(df_model)
    
    df_encoded = DataFrame(encoded_seq)
    
    df_model = df_model.join(df_encoded, how='right', lsuffix='', rsuffix='_ae', sort=False)
    
    df_model = change_index(df_model)
    
    df_model = add_class(df_model)
    
    return (df_model)
    

In [29]:
df_model = ACP()

ACP- Cantidad de Sparse-Matrix:  758 - Largo Sparse Matrix:  27
noACP- Cantidad de Sparse-Matrix:  758 - Largo Sparse Matrix:  27


In [30]:
df_model.shape

(1516, 1060)

In [31]:
df_model.to_csv('../Databases/Datasets with differents encoded types/No Feature/nf_polarizability.txt', header=None, index=None, sep=' ', mode='a')